## 1. Bagging Ensemble technique with random forest

In [28]:
import seaborn as sns
data = sns.load_dataset('tips')
data.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [29]:
data['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [30]:
data['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [31]:
data.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [32]:
from sklearn.preprocessing import LabelEncoder 

In [33]:
encoder = LabelEncoder()

data['time']=encoder.fit_transform(data['time'])

In [34]:
X = data.drop(labels=['time'],axis=1)
y= data['time']

In [35]:
X

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [36]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [37]:
X['day'].value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [38]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20, random_state=42)

In [39]:
X_train

,total_bill,tip,sex,smoker,day,size
228,13.28,2.72,Male,No,Sat,2
208,24.27,2.03,Male,Yes,Sat,2
96,27.28,4.00,Male,Yes,Fri,2
167,31.71,4.50,Male,No,Sun,4
84,15.98,2.03,Male,No,Thur,2
...,...,...,...,...,...,...
106,20.49,4.06,Male,Yes,Sat,2
14,14.83,3.02,Female,No,Sun,2
92,5.75,1.00,Female,Yes,Fri,2
179,34.63,3.55,Male,Yes,Sun,2


In [40]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer # missing data fillup
from sklearn.preprocessing import StandardScaler # scaling down values
from sklearn.preprocessing import OneHotEncoder # encoding categorical data
from sklearn.compose import ColumnTransformer # to implement pipeline in series..

In [41]:
cat_col = ['sex','smoker','day']
num_col = ['total_bill','tip','size']

In [45]:
num_pipeline=Pipeline(
    
    steps=[
        ("simpleimputer",SimpleImputer(strategy="median")),
        ("scaler",StandardScaler())
    ]    
)

cat_pipeline = Pipeline(
    steps=[
        ('simpleimputer',SimpleImputer(strategy="most_frequent")),
        ('onehotencoder',OneHotEncoder())
    ]
)

In [46]:
preprocessor = ColumnTransformer(
    [
        ("num_pipeline",num_pipeline, num_col),
        ("cat_pipeline",cat_pipeline, cat_col)
    ]
)

In [47]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [48]:
X_train

array([[-0.79306155, -0.2580329 , -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.46322744, -0.74211442, -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.80730659,  0.6399734 , -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.65383098, -1.46472887, -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.64749986,  0.32426806, -0.61214068, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.75289699, -0.41237773,  0.45363997, ...,  1.        ,
         0.        ,  0.        ]])

In [49]:
y_train

228    0
208    0
96     0
167    0
84     1
      ..
106    0
14     0
92     0
179    0
102    0
Name: time, Length: 195, dtype: int32

In [50]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [68]:
# mdoels as dictionary....
models = {
    "RandomForestClassifier":RandomForestClassifier(oob_score=True),
    "LogisticRegression":LogisticRegression(),
    "DecisionTreeClassifier":DecisionTreeClassifier()
    }

In [52]:
models.values()

dict_values([RandomForestClassifier(), LogisticRegression(), DecisionTreeClassifier()])

In [53]:
list(models.values())

[RandomForestClassifier(), LogisticRegression(), DecisionTreeClassifier()]

In [54]:
list(models.values())[0]

RandomForestClassifier()

In [55]:
models.keys()

dict_keys(['RandomForestClassifier', 'LogisticRegression', 'DecisionTreeClassifier'])

In [72]:
from sklearn.metrics import accuracy_score
def evaluate_model(X_train, y_train, X_test, y_test, models):
    
    report = {}
    
    for i in range(len(models)):
        
        mod = list(models.values())[i]
        
        mod.fit(X_train, y_train)
        
        y_pred = mod.predict(X_test)
        
        accuracy = round(accuracy_score(y_test, y_pred )* 100, 2)
        
        report[list(models.keys())[i]]=accuracy
        
    return report

In [74]:
evaluate_model(X_train, y_train, X_test, y_test, models)

{'RandomForestClassifier': 95.92,
 'LogisticRegression': 100.0,
 'DecisionTreeClassifier': 93.88}

In [76]:
params={
    "n_estimators":[50,100,200],
    "criterion":["gini","entropy"],
    "max_depth":[3,5,10]
}

In [80]:
model = RandomForestClassifier(oob_score=True)

In [77]:
from sklearn.model_selection import RandomizedSearchCV

In [85]:
cv = RandomizedSearchCV(model, param_distributions=params, scoring='accuracy',cv=5, verbose=3)

In [86]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.974 total time=   0.7s
[CV 2/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.923 total time=   0.4s
[CV 3/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=1.000 total time=   0.4s
[CV 4/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.949 total time=   0.3s
[CV 5/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.923 total time=   0.4s
[CV 1/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.923 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=10, n_estimators=100;, score=1.000 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.897 total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.923 total time=  

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(oob_score=True),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10],
                                        'n_estimators': [50, 100, 200]},
                   scoring='accuracy', verbose=3)

In [87]:
cv.best_params_

{'n_estimators': 200, 'max_depth': 10, 'criterion': 'entropy'}

In [90]:
model1 = RandomForestClassifier(n_estimators= 200, max_depth= 10, criterion= 'entropy',oob_score=True)

In [91]:
model1.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=200,
                       oob_score=True)

In [92]:
y_pred = model1.predict(X_test)
accuracy = round(accuracy_score(y_test, y_pred )* 100, 2)
accuracy # traininf accuracy...

95.92